In [1]:
#Please enter here the paths of the training file and the test file
training_file = "Untitled Folder/Datasets/seq2tree_atis/train.txt"
test_file = "Untitled Folder/Datasets/seq2tree_atis/test.txt"

"Untitled Folder/Datasets/seq2tree_atis/test.txt"

In [2]:
#Please enter here the paths of the training file and the test file
training_file = "Untitled Folder/Datasets/seq2tree_geoqueries/train.txt"
test_file = "Untitled Folder/Datasets/seq2tree_geoqueries/test.txt"

"Untitled Folder/Datasets/seq2tree_geoqueries/test.txt"

In [1]:
training_file = "Untitled Folder/Datasets/seq2tree_jobqueries/train.txt"
test_file = "Untitled Folder/Datasets/seq2tree_jobqueries/test.txt"

"Untitled Folder/Datasets/seq2tree_jobqueries/test.txt"

In [2]:
using Knet: Knet, AutoGrad, param, param0, mat, RNN, relu, Data, adam, progress, nll, zeroone
using Distributions
import .Iterators: cycle, Cycle, take
using IterTools
import CUDA
using CUDA

In [3]:
function tok_int(training_file)
    
    #global int2tok_input, int2tok_output
    f = open(training_file)
    
    tok2int_input = Dict{String,Int}() #keys: unique input tokens. values: id number of the token.
    int2tok_input = Vector{String}() #indices: numbers. values: tokens corresponding to those numbers.
    push!(int2tok_input, "<s>") #start token
    push!(int2tok_input, "</s>") #stop token
    tok2int_input["<s>"] = 1
    tok2int_input["</s>"] = 2
    
    tok2int_output = Dict{String,Int}() #keys: unique output tokens. values: id number of the token.
    int2tok_output = Vector{String}() #keys: numbers. values: tokens corresponding to those numbers.
    push!(int2tok_output, "<s>") #start token
    push!(int2tok_output, "</s>") #stop token
    tok2int_output["<s>"] = 1
    tok2int_output["</s>"] = 2
    
    while ! eof(f)
        
        seq = readline(f)
        seq = chomp(seq)
        input, output = split(seq, "\t")
        tokens = split(input, " ")
        
        for token in tokens
            if !haskey(tok2int_input, token)
                push!(int2tok_input, token)
                tok2int_input[token] = length(int2tok_input)
            end            
        end
        
        tokens = split(output, " ")
        
        for token in tokens
            if !haskey(tok2int_output, token)
                push!(int2tok_output, token)
                tok2int_output[token] = length(int2tok_output)
            end           
        end
        
    end
    
    push!(int2tok_input, "UNK") 
    tok2int_input["UNK"] = length(int2tok_input)
    push!(int2tok_output, "UNK") 
    tok2int_output["UNK"] = length(int2tok_output)
    
    @assert length(int2tok_input) == length(tok2int_input)
    @assert length(int2tok_output) == length(tok2int_output)
    
    Vq = length(int2tok_input) #number of unique input tokens
    Va = length(int2tok_output) #number of unique output tokens 
    
    return int2tok_input, tok2int_input, int2tok_output, tok2int_output, Va, Vq
    
end


tok_int (generic function with 1 method)

In [4]:
function data_reader(training_file, tok2int_input, tok2int_output)
    
    data = []
    
    f = open(training_file)
    
    while ! eof(f)
        
        seq = readline(f)
        seq = chomp(seq)
        input, output = split(seq, "\t")
        tokens = split(input, " ")
        
        src_tokens = Vector{Int}() #vector that stores the token ids.
        global n_in = 0 #just to check the number of unknown tokens
        
        for token in tokens
            
            if haskey(tok2int_input, token)
                push!(src_tokens, tok2int_input[token])
            else
                push!(src_tokens, tok2int_input["UNK"])
                n_in += 1
            end
            
        end        

        tokens = split(output, " ")
        tgt_tokens = Vector{Int}() #vector that stores the token ids.
        global n_out = 0 #just to check the number of unknown tokens
        
        for token in tokens
            
            if haskey(tok2int_output, token)
                push!(tgt_tokens, tok2int_output[token])
            else
                push!(tgt_tokens, tok2int_output["UNK"])
                n_out += 1
            end
            
        end 
        
        push!(data, (src_tokens, tgt_tokens))
        
    end    
    
    return data
    
end


data_reader (generic function with 1 method)

In [5]:
# ATIS: source sequence lengths are monotonically increasing as you read through the lines of the trn file.
# Not the same for the target sequences!

function minibatch(data, batchsize)
    
    n = length(data)
    batch_data = Any[]
    p = 0
    
    while p + batchsize <= n
        
        max_seq_len = length(data[p+batchsize][1])
        enc_seq = zeros(Int64, batchsize, max_seq_len+2) 
        enc_seq[:,max_seq_len+2] .= 2 #end of sequence token
        
        for i in 1:batchsize
            
            seq = data[p+i][1]
            seq_len = length(seq)
            
            for j in 1:seq_len
                enc_seq[i, max_seq_len-seq_len+j+1] = seq[seq_len-j+1]
            end
            
            for k in 1:(max_seq_len+1-seq_len) #pad the start with start of seq tokens
                enc_seq[i, k] = 1
            end
            
        end
        
        max_seq_len = -1
        
        for i in 1:batchsize
            
            seq = data[p+i][2]
            
            if length(seq) > max_seq_len 
                max_seq_len = length(seq)
            end
            
        end
        
        dec_seq = zeros(Int64, batchsize, max_seq_len+2)
        dec_seq[:,1] .= 1
        
        for i in 1:batchsize
            
            seq = data[p+i][2]
            seq_len = length(seq)
            
            for j in 1:seq_len
                dec_seq[i, (j+1)] = seq[j]
            end
            
            dec_seq[i, (seq_len+2):end] .= 2
            
        end
        
        p += batchsize
        
        push!(batch_data, (enc_seq, dec_seq))
        
    end
    
    return batch_data
    
end               
    

minibatch (generic function with 1 method)

In [6]:
function mask(a, pad)
    
    a = copy(a)
    
    for i in 1:size(a, 1)
        
        j = size(a,2)
        
        while a[i, j] == pad && j > 1
            
            if a[i, j - 1] == pad
                a[i, j] = 0
            end
            
            j -= 1
            
        end
        
    end
    
    return a
    
end

mask (generic function with 1 method)

In [7]:
struct Embed    
    w
end

function Embed(embedsize::Int, vocabsize::Int)
    
    w = rand(Uniform(-0.08,0.08), embedsize, vocabsize)
    w = Knet.Param(convert(Knet.KnetArray{Float32},w))
    
    return Embed(w)
    
end

(e::Embed)(x) = e.w[:,x] #x: word id

struct Linear
    w
    b
    f
end

Linear(i::Int,o::Int,f=identity) = Linear(param(o,i), param0(o), f)

(d::Linear)(x) = d.f.(d.w * mat(x,dims=1) .+ d.b)

In [8]:
struct seq2seq
    input_embed
    output_embed
    encoder
    decoder
    linear
    dropout
end

function seq2seq(X::Int, H::Int, Vq::Int, Va::Int, dropout::Real)
    
    a = Embed(X, Vq)
    b = Embed(X, Va)
    c = RNN(X, H; rnnType=:lstm, numLayers=1, dropout=dropout)
    d = RNN(X, H; rnnType=:lstm, numLayers=1, dropout=dropout)
    e = Linear(H,Va)
    f = dropout
    
    return seq2seq(a, b, c, d, e, f)
    
end

function (s::seq2seq)(x, y; average=true) 
    
    s.encoder.h = 0
    s.encoder.c = 0
    
    x_embedded = s.input_embed(x) # BURAYA DROPUT EKLE!
    y_embedded = s.output_embed(y) # BURAYA DROPUT EKLE!
    
    encoder_out = s.encoder(x_embedded)
    
    s.decoder.h = s.encoder.h
    s.decoder.c = s.encoder.c
    
    decoder_out = s.decoder(y_embedded[:, :, 1:end-1])
    
    dims = size(decoder_out)
    
    output = s.linear(Knet.dropout(reshape(decoder_out, dims[1], dims[2] * dims[3]), s.dropout))
    
    scores = reshape(output, size(output, 1), dims[2], dims[3])
    
    nll(scores, mask(y[:, 2:end], 2); dims=1, average=average)
    
end


In [9]:
### FOR PREDICTION/TESTING

function (s::seq2seq)(x; stopfactor = 4)
    
    s.encoder.h = 0
    s.encoder.c = 0
    
    x_embedded = s.input_embed(x)
    
    encoder_out = s.encoder(x_embedded)
    
    s.decoder.h = s.encoder.h
    s.decoder.c = s.encoder.c
    
    step = fill(1, size(x)[1], 1)
    out = fill(2, size(x)[1], 0)
    stopping_criteria = fill(false, size(x)[1])
    
    for i in 1:(stopfactor * size(x,2))
        
        if sum(stopping_criteria) == size(x)[1]
            break
        end
        
        step_embed = s.output_embed(step)
        
        decoder_out = s.decoder(step_embed)
        
        dims = size(decoder_out)
        
        output = s.linear(reshape(decoder_out, dims[1], dims[2] * dims[3]))
        
        step = transpose([i[1] for i in argmax(output; dims=1)])
        
        out = hcat(out, step)
        
        for e in findall(x -> x == 2, step)
            stopping_criteria[e[1]] = true
        end
        
    end
    
    return out
    
end

In [10]:
function model_accuracy(model, data)
    
    total = 0
    no_sequences = 0
    
    for (x, y) in data
        
        y_pred = model(x)
        
        for i in 1:size(x, 1)
            
            no_sequences += 1            
            y_clipped = []
            y_pred_clipped = []
            
            for j in 1:size(y,2)   
                
                if y[i,j] == 2                   
                    append!(y_clipped, y[i,2:j])
                    break
                end
                
            end
            
            for j in 1:size(y_pred,2)
                
                if y_pred[i,j] == 2 
                    append!(y_pred_clipped, y_pred[i,1:j])
                    break
                end     
               
            end
            
            if length(y_clipped) == length(y_pred_clipped)
                
                if sum(y_pred_clipped .== y_clipped) == size(y_clipped, 1)                   
                    total += 1                    
                end
                
            end
            
        end
        
    end
    
    return total / no_sequences
    
end

model_accuracy (generic function with 1 method)

In [11]:
int2tok_input, tok2int_input, int2tok_output, tok2int_output, Va, Vq = tok_int(training_file)

trndata = data_reader(training_file, tok2int_input, tok2int_output)
tstdata = data_reader(test_file, tok2int_input, tok2int_output)

X = 200
H = 200

trndata_batch = minibatch(trndata, 20)
tstdata_batch = minibatch(tstdata, 20)

22-element Array{Any,1}:
 ([1 1 … 69 2; 1 1 … 69 2; … ; 1 6 … 7 2; 1 157 … 6 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 8 2])
 ([1 157 … 6 2; 1 157 … 23 2; … ; 1 6 … 69 2; 1 76 … 14 2], [1 3 … 8 2; 1 3 … 8 2; … ; 1 3 … 2 2; 1 56 … 2 2])
 ([1 1 … 14 2; 1 1 … 14 2; … ; 1 145 … 6 2; 1 145 … 23 2], [1 56 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 1 … 6 2; 1 1 … 23 2; … ; 1 23 … 159 2; 1 23 … 69 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 42 … 14 2; 1 51 … 14 2; … ; 1 23 … 69 2; 1 23 … 69 2], [1 36 … 2 2; 1 15 … 2 2; … ; 1 3 … 8 2; 1 3 … 8 2])
 ([1 6 … 69 2; 1 23 … 69 2; … ; 1 75 … 81 2; 1 75 … 81 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 56 … 2 2; 1 56 … 2 2])
 ([1 1 … 69 2; 1 1 … 69 2; … ; 1 32 … 14 2; 1 70 … 69 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 1 … 14 2; 1 1 … 18 2; … ; 1 43 … 69 2; 1 6 … 69 2], [1 56 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 23 … 43 2; 1 23 … 14 2; … ; 1 34 … 69 2; 1 34 … 69 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3

In [12]:
my_model = seq2seq(X,H,Vq,Va,0.3) # dropout=0.4 for GEO and JOBS datasets, dropout = 0.3 for ATIS dataset.

seq2seq(Embed(P(Knet.KnetArrays.KnetArray{Float32,2}(200,436))), Embed(P(Knet.KnetArrays.KnetArray{Float32,2}(200,169))), LSTM(input=200,hidden=200,dropout=0.3), LSTM(input=200,hidden=200,dropout=0.3), Linear(P(Knet.KnetArrays.KnetArray{Float32,2}(169,200)), P(Knet.KnetArrays.KnetArray{Float32,1}(169)), identity), 0.3)

In [13]:
test_accs = []

for i in 1:200
    
    Knet.rmsprop!(my_model, trndata_batch;rho=0.95, gclip=5)
    
    if i%5 == 0
        println("After $i epochs")
        test_acc = model_accuracy(my_model, tstdata_batch)
        push!(test_accs, test_acc)
        println("test accuracy: ", test_acc)
    end
end

After 5 epochs
test accuracy: 0.045454545454545456
After 10 epochs
test accuracy: 0.1159090909090909
After 15 epochs
test accuracy: 0.15227272727272728
After 20 epochs
test accuracy: 0.6022727272727273
After 25 epochs
test accuracy: 0.6954545454545454
After 30 epochs
test accuracy: 0.7204545454545455
After 35 epochs
test accuracy: 0.7204545454545455
After 40 epochs
test accuracy: 0.7477272727272727
After 45 epochs
test accuracy: 0.7772727272727272
After 50 epochs
test accuracy: 0.775
After 55 epochs
test accuracy: 0.7727272727272727
After 60 epochs
test accuracy: 0.7931818181818182
After 65 epochs
test accuracy: 0.8022727272727272
After 70 epochs
test accuracy: 0.7909090909090909
After 75 epochs
test accuracy: 0.7977272727272727
After 80 epochs
test accuracy: 0.7977272727272727
After 85 epochs
test accuracy: 0.8204545454545454
After 90 epochs
test accuracy: 0.7840909090909091
After 95 epochs
test accuracy: 0.8113636363636364
After 100 epochs
test accuracy: 0.8068181818181818
After 105 